In [1]:
from scipy.spatial import ConvexHull
import numpy as np
import json 

In [10]:
file = '../src/Dem/Mesh-D4.json'
viewpoint=[0]  #viewpoint in the dimensions higher than 1-3 (1-based counting). Size d-3
outfile = 'MeshD4'

In [3]:
def almostequal (a, b):
    epsilon = 1e-7 ; 
    return abs(a-b)<=epsilon ; 

def segment_3dspace_intersection(vp, p1, p2):
    u=np.array(p2)-np.array(p1) ; 
    v=np.hstack(([0,0,0],vp)) ;
    ninside=0 ; 
    for i in range(3,len(p1)):
        if almostequal(u[i],0):
            if not almostequal(v[i], p1[i]): return [] ;
            else: ninside +=1 ; 
        else:
            alpha=(v[i]-p1[i])/u[i] ; 
            if alpha>1 or alpha<0 : return [] ;
            if i>3:
                if not almostequal(alpha, alphaprev): return [] ; 
            alphaprev=alpha ; 
    
    #we got one!
    if ninside==len(p1)-3: #all the segment is inside
        return [np.array(p1),np.array(p2)] ;
    else:
        inter=p1+alpha*u ;
        return [inter] ; 

def intersectionpoints(obj, vp):
    pts=[] 
    if obj['dimensionality']==0: return [np.array(obj['vertices'][0])] 
    for i in range (0, obj['dimensionality']+1):
        for j in range(i+1, obj['dimensionality']+1):
            pts += segment_3dspace_intersection(vp, obj['vertices'][i], obj['vertices'][j])
    return pts ; 

def normal (p1, p2, p3):
    u=p2-p1 ; 
    v=p3-p1 ; 
    w=np.cross(u,v) ; 
    w/=np.linalg.norm(w) ; 
    return w ; 

In [13]:
f = open(file)
data = json.load(f)
f.close() ; 

f = open(outfile+'.vtk', "w")
f.write("# vtk DataFile Version 2.0\nReally cool data\nASCII\nDATASET POLYDATA\n")

allpoints=np.empty((0,3)) ; 
allvertices=np.empty((0,1), dtype=np.int64)  ; 
alllines=np.empty((0,2), dtype=np.int64)  ; 
alltriangles=np.empty((0,3), dtype=np.int64)  ; 
npts=0 ; 

for o in data['objects']:
    pts=intersectionpoints(o, viewpoint);
    pts = [ pts[i][0:3] for i in range(0,len(pts))]
    pts = np.unique(np.array(pts), axis=0)
    
    if   len(pts)==0: continue ; 
    elif len(pts)==1:
        allpoints = np.vstack((allpoints, pts)) ;  
        allvertices = np.vstack((allvertices, npts)) ; 
        npts += 1 ; 
    elif len(pts)==2:
        allpoints = np.vstack((allpoints, pts)) ;  
        alllines = np.vstack((alllines, [npts, npts+1])) ;
        npts += 2 ; 
    elif len(pts)==3:
        allpoints = np.vstack((allpoints, pts)) ; 
        alltriangles = np.vstack((alltriangles, [npts, npts+1, npts+2])) ;
        npts += 3 ; 
    else :
        hull = ConvexHull(pts)
        allpoints = np.vstack((allpoints, pts)) ; 
        alltriangles = np.vstack((alltriangles, hull.simplices+npts)) ; 
        npts += len(pts) ; 
    
f.write(f"POINTS {len(allpoints)} float\n")
for i in allpoints:
    f.write(f"{i[0]} {i[1]} {i[2]} \n")

f.write(f"VERTICES {len(allvertices)} {len(allvertices)*2}\n")
for i in allvertices:
    f.write(f"1 {i[0]}\n")
    
f.write(f"LINES {len(alllines)} {len(alllines)*3}\n")
for i in alllines:
    f.write(f"2 {i[0]} {i[1]}\n")

f.write(f"POLYGONS {len(alltriangles)} {len(alltriangles)*4}\n")
for i in alltriangles:
    f.write(f"3 {i[0]} {i[1]} {i[2]}\n")
    
f.close() ; 



In [113]:
f = open(file)
data = json.load(f)
f.close() ; 

if data['dimension'] < 3 : 
    print('No possible conversion to stl for dimensions lower than 3.')

#assert data['dimension']==len(viewpoint)+3, "Number of dimensions in mesh file inconsistent with the viewpoint length."
f = open(outfile+'.stl', "w")
f.write("solid convertedfromjson\n")

for o in data['objects']:
    pts=intersectionpoints(o, viewpoint);
    pts = [ pts[i][0:3] for i in range(0,len(pts))]
    pts = np.unique(np.array(pts), axis=0)
    hull = ConvexHull(pts)
    
    for i in hull.simplices:
        n = normal(hull.points[i[0]], hull.points[i[1]], hull.points[i[2]])
        f.write(f"\tfacet normal {n[0]} {n[1]} {n[2]}\n")
        f.write("\touter loop\n") ;
        f.write(f"\t\tvertex {hull.points[i[0]][0]} {hull.points[i[0]][1]} {hull.points[i[0]][2]}\n")
        f.write(f"\t\tvertex {hull.points[i[1]][0]} {hull.points[i[1]][1]} {hull.points[i[1]][2]}\n")
        f.write(f"\t\tvertex {hull.points[i[2]][0]} {hull.points[i[2]][1]} {hull.points[i[2]][2]}\n")
        f.write("\tendloop\nendfacet\n")
    

f.write("endsolid convertedfromjson")
f.close() ; 





[[2 3 0]
 [1 3 0]
 [1 2 0]
 [1 2 3]]


In [12]:
f = open('../src/Dem/Mesh.json')
data = json.load(f)
f.close() ; 

In [124]:
rng = np.random.default_rng()
points = rng.random((4, 3))
hull = ConvexHull(points)
hull.simplices

array([[1, 0, 3],
       [2, 0, 3],
       [2, 1, 3],
       [2, 1, 0]], dtype=int32)

In [133]:
a=[] ; 
a+=[[1,2],[2,2]]
a+=[]
print(a)

[[1, 2], [2, 2]]


TypeError: unsupported operand type(s) for -: 'list' and 'list'